In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import pandasql as ps


In [3]:
path = os.getenv('path')
df = pd.read_csv(path + 'data_used.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   referral_id         1470 non-null   int64 
 1   creation_date       1470 non-null   object
 2   creation_time       1470 non-null   object
 3   status_date         1470 non-null   object
 4   status_time         1470 non-null   object
 5   company_id          1470 non-null   int64 
 6   partner_id          1470 non-null   int64 
 7   consultant_id       1470 non-null   int64 
 8   status              1470 non-null   object
 9   is_outbound         1470 non-null   int64 
 10  partner_type        1470 non-null   object
 11  lead_sales_contact  1470 non-null   object
 12  country             1113 non-null   object
dtypes: int64(5), object(8)
memory usage: 149.4+ KB


In [4]:
df.head()

,referral_id,creation_date,creation_time,status_date,status_time,company_id,partner_id,consultant_id,status,is_outbound,partner_type,lead_sales_contact,country
0,1352,2021-04-20,08:51:23 PM,2021-04-20,08:51:23 PM,1177,376,518,successful,0,IFA,Leaf,UK
1,452,2021-01-07,05:53:25 PM,2021-01-07,05:53:25 PM,816,147,207,successful,0,IFA,Leaf,UK
2,26,2020-09-15,05:04:10 AM,2020-09-15,05:04:10 AM,0,10,17,successful,0,IFA,Apple,Singapore
3,169,2020-10-21,02:20:15 AM,2020-10-21,02:20:15 AM,608,10,17,successful,0,IFA,Apple,Singapore
4,1455,2021-05-02,11:14:59 PM,2021-05-02,11:14:59 PM,0,14,26,pending,0,IFA,Apple,Singapore


In [17]:
q1 = """ SELECT 
         DISTINCT status,
         COUNT(DISTINCT referral_id) AS count_referrals,
         strftime('%Y', status_date) AS year
    FROM df
    WHERE year = '2021'
    GROUP BY
    status

ORDER BY COUNT(DISTINCT referral_id) DESC
"""
print(ps.sqldf(q1, locals()))

          status  count_referrals  year
0        pending              517  2021
1     successful              468  2021
2  disinterested               41  2021


In [6]:
q2 = """ SELECT 
         --status_date,
         COUNT(DISTINCT referral_id) AS count_referrals,
         strftime('%Y', status_date) AS year,
         strftime('%m', status_date) AS month
    FROM df
    GROUP BY month

    ORDER BY year DESC, month ASC
"""
print(ps.sqldf(q2, locals()))

   count_referrals  year month
0              109  2021    01
1              213  2021    02
2              406  2021    03
3              280  2021    04
4               18  2021    05
5               81  2020    09
6              178  2020    10
7              119  2020    11
8               66  2020    12


In [8]:
q3 = """ SELECT 
         partner_type,   
         COUNT(DISTINCT partner_id) AS count_partners,
         strftime('%Y', status_date) AS year
         --strftime('%m', status_date) AS month
    FROM df
    WHERE year = '2021'
    GROUP BY partner_type

    ORDER BY count_partners DESC
"""
print(ps.sqldf(q3, locals()))

         partner_type  count_partners  year
0               Agent              57  2021
1                 IFA              10  2021
2               Other               9  2021
3          Influencer               5  2021
4              Lender               3  2021
5  Management company               2  2021
6             Insurer               1  2021
7           Developer               1  2021


In [19]:
q4 = """ SELECT 
         COUNT(DISTINCT partner_id) AS count_partners,   
         COUNT(DISTINCT referral_id) AS count_referrals,
         COUNT(DISTINCT company_id) AS count_companies,
         COUNT(DISTINCT lead_sales_contact) AS count_sales_people,
         
         strftime('%Y', status_date) AS year
         --strftime('%m', status_date) AS month
    FROM df
    WHERE year = '2021' 
    --GROUP BY year
"""
print(ps.sqldf(q4, locals()))

   count_partners  count_referrals  count_companies  count_sales_people  year
0              88             1026              473                   8  2021


In [24]:
q5 = """ SELECT 
         DISTINCT partner_id,   
         COUNT(DISTINCT referral_id) AS count_referrals,
         strftime('%Y', status_date) AS year
         
    FROM df
    WHERE year = '2021' 

    GROUP BY partner_id

    ORDER BY count_referrals DESC

    LIMIT 10
"""
print(ps.sqldf(q5, locals()))

   partner_id  count_referrals  year
0         191              130  2021
1         149              115  2021
2          63               86  2021
3         201               85  2021
4         290               78  2021
5           7               76  2021
6         302               60  2021
7         213               35  2021
8          22               33  2021
9          21               27  2021


In [31]:
q6 = """ SELECT 
         COUNT(DISTINCT company_id) AS count_companies,
         strftime('%Y', status_date) AS year
         
    FROM df
    WHERE year = '2021' and is_outbound = 1
"""
print(ps.sqldf(q6, locals()))

   count_companies  year
0              182  2021
